In [299]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

from sklearn.metrics import f1_score, confusion_matrix, accuracy_score, classification_report, precision_score, recall_score
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


from sklearn.preprocessing import StandardScaler,LabelEncoder,OneHotEncoder,OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline

In [300]:
data = pd.read_csv(r'D:\customer\customer_churn\Notebook\data\WA_Fn-UseC_-Telco-Customer-Churn.csv')
data.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [301]:

data['TotalCharges']=data['TotalCharges'].replace(" ",np.nan)
data=data[data['TotalCharges'].notnull()]


In [302]:
data = data.reset_index()[data.columns]
data['TotalCharges'] = data['TotalCharges'].astype(float)

In [303]:
# checking the unique values of the clumns
for col in data.describe(include='all').columns:
    print(col)
    print(data[col].unique())
    print('*'*30)

customerID
['7590-VHVEG' '5575-GNVDE' '3668-QPYBK' ... '4801-JZAZL' '8361-LTMKD'
 '3186-AJIEK']
******************************
gender
['Female' 'Male']
******************************
SeniorCitizen
[0 1]
******************************
Partner
['Yes' 'No']
******************************
Dependents
['No' 'Yes']
******************************
tenure
[ 1 34  2 45  8 22 10 28 62 13 16 58 49 25 69 52 71 21 12 30 47 72 17 27
  5 46 11 70 63 43 15 60 18 66  9  3 31 50 64 56  7 42 35 48 29 65 38 68
 32 55 37 36 41  6  4 33 67 23 57 61 14 20 53 40 59 24 44 19 54 51 26 39]
******************************
PhoneService
['No' 'Yes']
******************************
MultipleLines
['No phone service' 'No' 'Yes']
******************************
InternetService
['DSL' 'Fiber optic' 'No']
******************************
OnlineSecurity
['No' 'Yes' 'No internet service']
******************************
OnlineBackup
['Yes' 'No' 'No internet service']
******************************
DeviceProtection
['No' 'Yes' 'No 

In [304]:
def replace_data(column):
    column = column.str.replace('No internet service', 'No')
    column = column.str.replace('No phone service', 'No')
    return column

In [305]:
data['MultipleLines'] = replace_data(data['MultipleLines'])
data['OnlineSecurity'] = replace_data(data['OnlineSecurity'])
data['OnlineBackup'] = replace_data(data['OnlineBackup'])
data['DeviceProtection'] = replace_data(data['DeviceProtection'])
data['TechSupport'] = replace_data(data['TechSupport'])
data['StreamingTV'] = replace_data(data['StreamingTV'])
data['StreamingMovies'] = replace_data(data['StreamingMovies'])

In [306]:
for i in data.describe(include=object).columns:
    print(i)
    print(data[i].unique())
    print("*"*30)


customerID
['7590-VHVEG' '5575-GNVDE' '3668-QPYBK' ... '4801-JZAZL' '8361-LTMKD'
 '3186-AJIEK']
******************************
gender
['Female' 'Male']
******************************
Partner
['Yes' 'No']
******************************
Dependents
['No' 'Yes']
******************************
PhoneService
['No' 'Yes']
******************************
MultipleLines
['No' 'Yes']
******************************
InternetService
['DSL' 'Fiber optic' 'No']
******************************
OnlineSecurity
['No' 'Yes']
******************************
OnlineBackup
['Yes' 'No']
******************************
DeviceProtection
['No' 'Yes']
******************************
TechSupport
['No' 'Yes']
******************************
StreamingTV
['No' 'Yes']
******************************
StreamingMovies
['No' 'Yes']
******************************
Contract
['Month-to-month' 'One year' 'Two year']
******************************
PaperlessBilling
['Yes' 'No']
******************************
PaymentMethod
['Electronic che

In [307]:
from sklearn.utils import resample


def upsample_minority_class(data, minority_class_label, majority_class_label):
    minority = data[data['Churn'] == minority_class_label]
    majority = data[data['Churn'] == majority_class_label]

    minority_upsampled = resample(minority, replace=True, n_samples=majority.shape[0])

    upsampled_data = pd.concat([minority_upsampled, majority], axis=0)

    return upsampled_data

In [308]:
data = upsample_minority_class(data, 'Yes', 'No')


In [309]:
data.shape

(10326, 21)

In [310]:
# Seprating the independet & dependent feature 

X = data.drop(columns=['Churn','customerID'])
y = data['Churn']

In [311]:
# seperating the numeric and categorical data and creating columns transformers 
num_featurs = X.select_dtypes(exclude='object').columns
cat_featurs = X.select_dtypes(include='object').columns

In [312]:
num_featurs

Index(['SeniorCitizen', 'tenure', 'MonthlyCharges', 'TotalCharges'], dtype='object')

In [313]:
cat_featurs

Index(['gender', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines',
       'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
       'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract',
       'PaperlessBilling', 'PaymentMethod'],
      dtype='object')

In [314]:
# initializing the transformer
numeric_transformer = StandardScaler()
encoder = OrdinalEncoder()

# creating colum transformer 

preprocessor = ColumnTransformer(
    [("Onehotencoding",encoder,cat_featurs),
    ("StandardScaler",numeric_transformer,num_featurs)])


In [315]:
X = preprocessor.fit_transform(X)

In [316]:
X.shape

(10326, 19)

In [317]:
# encoding the features also
label_encoder = LabelEncoder()
y=label_encoder.fit_transform(y)

In [318]:
y.shape

(10326,)

In [319]:
# seprating the dataset into train and test

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25,random_state=42)

In [320]:
print("X_train shappe",X_train.shape)
print("X_train shappe",X_test.shape)
print("y_train shappe",y_train.shape)
print("y_test shappe",y_test.shape)

X_train shappe (7744, 19)
X_train shappe (2582, 19)
y_train shappe (7744,)
y_test shappe (2582,)


In [321]:
models = [("logistic regressionn" , LogisticRegression()),
          ("KNN classifier" , KNeighborsClassifier()),
          ("Decesion tree classifier" , DecisionTreeClassifier()),
          ("Random classifier" , RandomForestClassifier()),
          ("Ada boost classifier" , AdaBoostClassifier()),
          ("gradient boosting classifier" , GradientBoostingClassifier()),
]

In [322]:
for name , model in models:
    print(f"Trainning Model : {name}")
    model.fit(X_train,y_train)

# Make predictions and evaluate each model using confusion matrices

for name , model in models:
    print(f"Evaluating model: {name}")
    y_pred =model.predict(X_test)
    cm = confusion_matrix(y_test, y_pred)
    print(f"Confusion matrix:\n{cm}")
    report = classification_report(y_test, y_pred)
    print(f"Classification report:\n{report}")

Trainning Model : logistic regressionn
Trainning Model : KNN classifier
Trainning Model : Decesion tree classifier
Trainning Model : Random classifier
Trainning Model : Ada boost classifier
Trainning Model : gradient boosting classifier
Evaluating model: logistic regressionn
Confusion matrix:
[[ 922  343]
 [ 266 1051]]
Classification report:
              precision    recall  f1-score   support

           0       0.78      0.73      0.75      1265
           1       0.75      0.80      0.78      1317

    accuracy                           0.76      2582
   macro avg       0.77      0.76      0.76      2582
weighted avg       0.76      0.76      0.76      2582

Evaluating model: KNN classifier
Confusion matrix:
[[ 845  420]
 [ 188 1129]]
Classification report:
              precision    recall  f1-score   support

           0       0.82      0.67      0.74      1265
           1       0.73      0.86      0.79      1317

    accuracy                           0.76      2582
   macro a